In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf
from django.db.models import Q
from apps.environment_simulator.sevice_layer.market_simulator import MarketSimulator
from apps.environment_simulator.models import SimulatedStockBuffer
import gymnasium as gym
import ray
from gymnasium import spaces
from ray.rllib.algorithms import ppo
from os import path, environ
from gymnasium.envs.registration import register

2023-10-08 11:51:31,725	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:

ray.shutdown()
ray.init(runtime_env={"working_dir": "/Users/mirbilal/Desktop/MobCommission/commissionV2/"})

# register(
#      id='MarketSimulator-v0',
#      entry_point='apps.environment_simulator.service_layer.market_simulator:MarketSimulator',
#      max_episode_steps=10,
# )


db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}

the_current_time_step  = datetime(year=1995, month=1, day=1, hour=10)

config = {
        "env": MarketSimulator,
        "env_config": {
            "db_params": db_params, 
            "max_episode_steps": 150, 
            "the_current_time_step": the_current_time_step,
            "print_output": False
        },
        "num_workers": 4,
}

algo = ppo.PPO(config=config)

2023-10-08 11:51:37,985	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-08 11:51:38,372	INFO packaging.py:518 -- Creating a file package for local directory '/Users/mirbilal/Desktop/MobCommission/commissionV2/'.
2023-10-08 11:51:38,630	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_9c3394e9fd8ca2d3.zip' (6.66MiB) to Ray cluster...
2023-10-08 11:51:38,660	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_9c3394e9fd8ca2d3.zip'.
2023-10-08 11:51:40,190	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-pack

ValueError: unconverted data remains: +00:00

In [6]:
mean_ppo = []
for _ in range(50):

    result = algo.train()
    print("episode reward mean:", _, result['episode_reward_mean'])
    mean_ppo.append(result['episode_reward_mean'])


episode reward mean: 0 -13741435.702940563
episode reward mean: 1 -13778807.85686907
episode reward mean: 2 -13805652.885868624
episode reward mean: 3 -13832426.082810521
episode reward mean: 4 -13837992.864861565
episode reward mean: 5 -13807646.120933339
episode reward mean: 6 -13765412.20428003
episode reward mean: 7 -13696589.678287908
episode reward mean: 8 -13666744.964022469
episode reward mean: 9 -13646368.824522365
episode reward mean: 10 -13609524.841649031
episode reward mean: 11 -13542905.511944456
episode reward mean: 12 -13510343.129069565
episode reward mean: 13 -13457235.053133197
episode reward mean: 14 -13463061.807736322
episode reward mean: 15 -13524090.24355485
episode reward mean: 16 -13523599.872955132
episode reward mean: 17 -13522383.309836216
episode reward mean: 18 -13519605.809652073
episode reward mean: 19 -13423259.077064088
episode reward mean: 20 -13364364.288625665
episode reward mean: 21 -13330600.77965687
episode reward mean: 22 -13273535.040921068
ep

OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


episode reward mean: 26 -13197859.517555358
episode reward mean: 27 -13189056.861718262
episode reward mean: 28 -13215163.38402251
episode reward mean: 29 -13199270.540891418
episode reward mean: 30 -13225611.427590353
episode reward mean: 31 -13262206.899838476


OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


episode reward mean: 32 -13218559.28428762
episode reward mean: 33 -13185689.292075342
episode reward mean: 34 -13149331.13998796
episode reward mean: 35 -13135132.38392702
episode reward mean: 36 -13126123.672777817
episode reward mean: 37 -13176576.54993772
episode reward mean: 38 -13213345.299289603
episode reward mean: 39 -13222790.89647089
episode reward mean: 40 -13181214.400171354
episode reward mean: 41 -13193980.610254195
episode reward mean: 42 -13133960.294170406
episode reward mean: 43 -13122254.856816437
episode reward mean: 44 -13112130.659900744
episode reward mean: 45 -13062978.1924536
episode reward mean: 46 -13086676.096559603
episode reward mean: 47 -13113638.816259319
episode reward mean: 48 -13142113.24052751
episode reward mean: 49 -13130282.145371566


In [7]:
my_current_time_step  = datetime(year=1995, month=1, day=1, hour=10)

config = {
    "db_params": db_params, 
    "max_episode_steps": 10, 
    "print_output": True,
    "the_current_time_step": my_current_time_step
}

myEnv = MarketSimulator(config)

def get_latest_states():
    latest_states = []
    try:
        env = myEnv
        env.reset()
        obs = env.state
        latest_states = [obs]  # To store the latest states from each worker
        for _ in range(10):  # Interact with the environment for 10 steps as an example
            action = algo.compute_single_action(obs)  # Replace with your agent's action
            obs, _, done, _ = env.step(action)
            if done:
                obs = env.reset()
            state = env.state  # Obtain the state from the environment
            latest_states.append(state)
    except Exception as e:
        print(e)
    return latest_states

latest_states = get_latest_states()
for a_state in latest_states:
    print(a_state)
    print("")


stepping_a [ 4.1814966  -0.02220719  1.7748228  -1.9925549  -7.738263  ]
[[1.57822450e+02]
 [3.10660832e+00]
 [4.09891726e+02]
 [8.03168974e+03]
 [2.80655342e+04]
 [3.72587265e+03]]
[[  87.40602652 1050.52535324  513.30215289   87.4152659   542.74283225
    87.42130536    0.           10.        ]
 [  24.88796476 4085.82720641 2159.98151224   24.89193074 2185.64775091
    24.89968625    0.           10.        ]
 [ 162.1910232  3150.27605446 1631.26782715  162.20821719 1425.56075021
   162.25684778    0.           10.        ]
 [  37.40892211 2451.738319   1217.04005846   37.41010749 1109.92991571
    37.41508652    0.           10.        ]
 [  21.68100633 4404.2458886  2227.88930773   21.69219909 2360.39860704
    21.71976       0.           10.        ]]
change_in_shares for index: 1 -0.022207186 0.0 2159.9815122390487
break_3
change_in_shares for index: 3 -1.9925549 0.0 1217.0400584616184
break_3
change_in_shares for index: 4 -7.738263 0.0 2227.8893077266875
break_3
change_in_shar